# Päringulaiendaja sisendandmebaasi valideerimine

Seda sammu pole otseselt vaja, kuid see võimaldab avastada andmetest tulenevaid ootamatusi.  

In [29]:
import sqlite3

from pandas import DataFrame

## I. Sõnavormide ning lemmade kuju valideerimine

Riigi Teataja seaduste ja määruste pealkirjade analüüsist (vaata skripti [`02A_validate_and_clean_source_texts_naively.ipynb`](../01_dokumentide_indekseerimine/02A_validate_and_clean_source_texts_naively.ipynb)) on meile teada kõik tekstides esinevad sümbolid. 
Indeksis olevatel sõnades peaks neis esinema vaid tähesümbolid, numbrid, sidekriipsud ja apostroofid. Vaatme kas see on ka tõsi. 

In [4]:
ESTONIAN_LETTER = '[a-z|öäõüžš]'
FOREIGN_LETTER = '[ôíëаa]'
NUMBER_SYMBOLS = '[0-9]'

# Tühikutega eraldatavad sümbolid 
PUNCTUATION_MARK = '[\\.,:;!?¿\\(\\)«»„““ˮ"‟”]'
SPECIAL_SYMBOLS = '[§/%\^]'
SUPERSCRIPT_SYMBOLS = '[⁰¹²³⁴⁵⁶⁷⁸⁹⁺⁻]'
SUBSCRIPT_SYMBOLS = '[₀₁₂₃₄₅₆₇₈₉₊₋]'

# Tühikutega asendatavad sümbolid  
WHITESPACE_SYMBOLS = '[\u200e\ufeff]'

# Teised lubatud sümbolid
DASH_SYMBOLS = '[‑-−‒-]'
OTHER_SYMBOLS = '[§/−a\^]'

In [33]:
con = sqlite3.connect('../results/query_extender_setup/rt_titles.sqlite')
cur = con.cursor()

result = cur.execute('SELECT DISTINCT vorm FROM  lemma_korpuse_vormid')
wordforms = DataFrame({'wordform':list(row[0] for row in result.fetchall())})

VALID_SYMBOL = f'{ESTONIAN_LETTER}|{FOREIGN_LETTER}|{DASH_SYMBOLS}|{NUMBER_SYMBOLS}'
idx = wordforms['wordform'].str.match(f'^{VALID_SYMBOL}+$', case=False)
display(wordforms[~idx])

,wordform


In [36]:
result = cur.execute('SELECT DISTINCT lemma FROM  lemma_korpuse_vormid')
lemmas = DataFrame({'lemma':list(row[0] for row in result.fetchall())})

VALID_SYMBOL = f'{ESTONIAN_LETTER}|{FOREIGN_LETTER}|{DASH_SYMBOLS}|{NUMBER_SYMBOLS}'
idx = lemmas['lemma'].str.match(f'^{VALID_SYMBOL}+$', case=False)
display(lemmas[~idx])

,lemma


## II. Liitsõnade tabeli valideerimne

Tehniliselt on kasulik kontrollida, et iga liitsõna tabelis olev liitlemma on ka pealkirjade tekstides esindatud.

In [40]:
result = cur.execute(
    """
    SELECT liitlemma FROM liitsõnad
    LEFT JOIN lemma_korpuse_vormid
    ON liitsõnad.liitlemma = lemma_korpuse_vormid.lemma
    where lemma is NULL"""    
)

exeptions =  DataFrame({'lemma':list(row[0] for row in result.fetchall())})
display(exeptions)


,lemma


In [41]:
con.close()